In [11]:
import cirq
import random
import matplotlib.pyplot as plt
import numpy as np

In [12]:
def make_quantum_teleportation_circuit(gate):
    """Returns a circuit for quantum teleportation.

    This circuit 'teleports' a random qubit state prepared by
    the input gate from Alice to Bob.
    """
    circuit = cirq.Circuit()

    # Get the three qubits involved in the teleportation protocol.
    msg = cirq.NamedQubit("Message")
    alice = cirq.NamedQubit("Alice")
    bob = cirq.NamedQubit("Bob")

    # The input gate prepares the message to send.
    circuit.append(gate(msg))

    # Create a Bell state shared between Alice and Bob.
    circuit.append([cirq.H(alice), cirq.CNOT(alice, bob)])

    # Bell measurement of the Message and Alice's entangled qubit.
    circuit.append([cirq.CNOT(msg, alice), cirq.H(msg), cirq.measure(msg, alice)])

    # Uses the two classical bits from the Bell measurement to recover the
    # original quantum message on Bob's entangled qubit.
    circuit.append([cirq.CNOT(alice, bob), cirq.CZ(msg, bob)])

    return circuit

-------------------------------
# Prueba 1:

(puerta: cirq.X ** 0.25)

In [18]:
"""Visualize the teleportation circuit."""
# Gate to put the message qubit in some state to send.
gate = cirq.X ** 0.25

# Create the teleportation circuit.
circuit1 = make_quantum_teleportation_circuit(gate)
print("Teleportation circuit:\n")
print(circuit1)


Teleportation circuit:

Alice: ─────H────────@───X───────M───@───────
                     │   │       │   │
Bob: ────────────────X───┼───────┼───X───@───
                         │       │       │
Message: ───X^0.25───────@───H───M───────@───


In [22]:
"""Display the Bloch vector of the message qubit."""
message = cirq.Circuit(gate.on(cirq.NamedQubit("Message"))).final_state_vector()
message_bloch_vector = cirq.bloch_vector_from_state_vector(message, index=0)
print("Bloch vector of message qubit before teleportation:")
print(np.round(message_bloch_vector, 3))


Bloch vector of message qubit before teleportation:
[ 0.    -0.707  0.707]


In [23]:
"""Simulate the teleportation circuit and get the final state of Bob's qubit."""
# Get a simulator.
sim = cirq.Simulator()

# Simulate the teleportation circuit.
result = sim.simulate(circuit1)

# Get the Bloch vector of Bob's qubit.
bobs_bloch_vector = cirq.bloch_vector_from_state_vector(result.final_state_vector, index=1)
message_bloch_vector2 = cirq.bloch_vector_from_state_vector(result.final_state_vector, index=0)
print("Bloch vector of Bob's qubit after teleportation:")
print(np.round(bobs_bloch_vector, 3))
print("Bloch vector of message qubit after teleportation:")
print(np.round(message_bloch_vector2, 3))

# Verify they are the same state!
np.testing.assert_allclose(bobs_bloch_vector, message_bloch_vector, atol=1e-7)


Bloch vector of Bob's qubit after teleportation:
[ 0.    -0.707  0.707]
Bloch vector of message qubit after teleportation:
[0. 0. 1.]


### Comentarios: 

Como podemos ver el estado del cubit Message (que es el que quiere mandar Alice a Bob) antes de realizar la teleportación cuántica es igual al estado del cubit de Bob despues de realizarla.

Tambien podemos apreciar que tras hacer el protocolo, Alice pierde el estado de su cubit, es decir tras la teleportación cuántica sigue quedando solo un cubit con ese estado (no podemos clonar el cubit) pero esta vez es Bob el que se ha quedado el cubit habiendo "teletransportado" el estado del cubit de Alice al cubit de Bob.

-------------------------------
# Prueba 2:

Vamos a hacer otra prueba aplicandole otra puerta al bit que queremos "teletransportar" para comprobar a ver si se sigue cumpliendo.

(puerta: ______________)

In [26]:
"""Visualize the teleportation circuit."""
# Gate to put the message qubit in some state to send.
gate = cirq.H

# Create the teleportation circuit.
circuit1 = make_quantum_teleportation_circuit(gate)
print("Teleportation circuit:\n")
print(circuit1)


Teleportation circuit:

Alice: ─────H───@───X───────M───@───────
                │   │       │   │
Bob: ───────────X───┼───────┼───X───@───
                    │       │       │
Message: ───H───────@───H───M───────@───


In [30]:
"""Display the Bloch vector of the message qubit."""
message = cirq.Circuit(gate.on(cirq.NamedQubit("Message"))).final_state_vector()
message_bloch_vector = cirq.bloch_vector_from_state_vector(message, index=0)
print("Bloch vector of message qubit before teleportation:")
print(np.round(message_bloch_vector, 3))


Bloch vector of message qubit before teleportation:
[1. 0. 0.]


In [31]:
"""Simulate the teleportation circuit and get the final state of Bob's qubit."""
# Get a simulator.
sim = cirq.Simulator()

# Simulate the teleportation circuit.
result = sim.simulate(circuit1)

# Get the Bloch vector of Bob's qubit.
bobs_bloch_vector = cirq.bloch_vector_from_state_vector(result.final_state_vector, index=1)
message_bloch_vector2 = cirq.bloch_vector_from_state_vector(result.final_state_vector, index=0)
print("Bloch vector of Bob's qubit after teleportation:")
print(np.round(bobs_bloch_vector, 3))
print("Bloch vector of message qubit after teleportation:")
print(np.round(message_bloch_vector2, 3))

# Verify they are the same state!
np.testing.assert_allclose(bobs_bloch_vector, message_bloch_vector, atol=1e-7)


Bloch vector of Bob's qubit after teleportation:
[1. 0. 0.]
Bloch vector of message qubit after teleportation:
[0. 0. 1.]


### Comentarios: 

Como vemos aqui se sigue cumpliendo lo mismo de antes, el cubit de Bob se ha quedado con el estado del cubit message y el cubit message ha colapsado por la medida que hemos hecho en el protocolo (en este caso ha colapsado a 0) y ha perdido su estado.